In [1]:
import pandas as pd
import re
df_t = pd.read_csv('transcript_all.csv')
df_p = pd.read_csv('product_all.csv')

# Match product name from url

In [2]:
def findnum(s):
    try:
        return re.findall(r'\d+', str(s))[0]
    except:
        return ''

In [3]:
df_t['tour_group_id']  = df_t.chat_start_url.apply(findnum)

In [4]:
tourid = df_p.loc[df_p.language_code == 'EN',['tour_group_id','name']]
tourid['tour_group_id'] = tourid.tour_group_id.apply(str)
df_m = pd.merge(df_t, tourid, on='tour_group_id')
df_text = df_m.groupby(['chat_id','name']).text.apply(' '.join).reset_index()

In [6]:
df_text.head()

,chat_id,name,text
0,P35ZGPQB35,Abu Dhabi & Dubai Big Bus Hop-On Hop-Off Combo,Hello. How may I help you? Hello. How may I he...
1,P35ZLP6FB5,Sentosa 4D Adventureland: Admission,Hello. How may I help you? Hello. How may I he...
2,P36ZUQLODI,SpongeBob SquarePants,Hello. How may I help you? Hi! How are! My chi...
3,P38Z2S52SH,Mean Girls,Hello. How may I help you? What are the dates ...
4,P38ZDSBEAP,Dubai City Tour & Burj Khalifa Tickets Combo,Hello. How may I help you? Hi Phil i had an en...


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=700)
t = tv.fit_transform(df_text.name)
terms = pd.DataFrame({'term': tv.get_feature_names(), 'tfidf': t.toarray().sum(0)})
t.shape
# terms.sort_values('tfidf', ascending=False)

(4299, 2369)

In [47]:
x = tv.transform(df_text.text)

In [48]:
from sklearn.preprocessing import LabelEncoder

In [49]:
le = LabelEncoder()

In [50]:
y = le.fit_transform(df_text.name)

In [51]:
y.shape

(4299,)

In [52]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [90]:
%%time
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_depth': 20,
    'random_state': 42
}
rf = RandomForestClassifier(**rf_params)

yp = cross_val_predict(rf, t, y, cv=5, method='predict_proba')

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:768: RuntimeWarning: Number of classes in training fold (353) does not match total number of classes (594). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  RuntimeWarning)


CPU times: user 24.4 s, sys: 2.37 s, total: 26.7 s
Wall time: 8.75 s


In [95]:
%%time
rf_params = {
    'n_jobs': -1,
    'n_estimators': 600,
    'max_depth': 100,
    'random_state': 42
}
rf = RandomForestClassifier(**rf_params)

rf.fit(x, y)

CPU times: user 52.6 s, sys: 1.77 s, total: 54.4 s
Wall time: 14.7 s


In [55]:
from sklearn.metrics import accuracy_score

In [96]:
%%time
accuracy_score(rf.predict(x), y)

CPU times: user 22.2 s, sys: 8.78 s, total: 31 s
Wall time: 8.19 s


0.8064666201442195

# Save model

In [101]:
from sklearn.externals import joblib

filename = 'product_RFmodel.sav'
joblib.dump(rf, filename)



['product_RFmodel.sav']

In [100]:
le.inverse_transform([rf.predict(tv.transform(['cats the musical']))])

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([['A Bronx Tale']], dtype=object)

In [95]:
plt.figure(figsize=(8,6))
fpr, tpr, _ = roc_curve(y, yp[:,1], average='micro')
auc = roc_auc_score(ytr, yp[:,1])
plt.plot(fpr, tpr, label='AUC=%.2f'%(auc))
plt.plot([0,1],[0,1],'--')
plt.axis([0,1,0,1])
plt.xlabel('1 - specificity',fontsize=14)
plt.ylabel('sensitivity',fontsize=14)
plt.legend(loc='lower right',fontsize=12)
plt.show()

TypeError: roc_curve() got an unexpected keyword argument 'average'

<Figure size 576x432 with 0 Axes>

In [97]:
roc_auc_score(y, yp[:,1],average='micro')

ValueError: multiclass format is not supported